In [ ]:
import os
import yaml
import sys
import json
import csv
from pathlib import Path

# Add src directory to Python path
src_path = Path.cwd() / 'src'
sys.path.insert(0, str(src_path))

from dotenv import load_dotenv
from nl_pe.llm import LLM_CLASSES
from nl_pe.llm.prompter import Prompter

In [ ]:
# Load configuration
config_path = Path('configs/llm/config.yaml')
with open(config_path, 'r') as config_file:
    config = yaml.safe_load(config_file)

load_dotenv()

# Instantiate the Prompter
prompter = Prompter(config)

In [ ]:
# Prompt with the template
template_path = 'hotel_gen.jinja2'
response = prompter.prompt_from_temp(template_path)

print("Full response:", response)

# Parse JSON response
if 'JSON_dict' in response:
    json_data = response['JSON_dict']
    print("Successfully parsed JSON with", len(json_data.get('doc_list', [])), "documents")
else:
    print("No JSON_dict found in response")
    json_data = None

In [ ]:
# Create CSV file from parsed JSON data
if json_data and 'doc_list' in json_data:
    # Create data/gen_docs directory if it doesn't exist
    output_dir = Path('data/gen_docs')
    output_dir.mkdir(parents=True, exist_ok=True)
    
    csv_path = output_dir / 'hotels_1.csv'
    
    with open(csv_path, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        # Write header
        writer.writerow(['d_id', 'd_text'])
        
        # Write data rows
        for i, doc_text in enumerate(json_data['doc_list'], 1):
            writer.writerow([i, doc_text])
    
    print(f"CSV file created successfully: {csv_path}")
    print(f"Total documents saved: {len(json_data['doc_list'])}")
else:
    print("No data to save to CSV - either JSON parsing failed or no doc_list found")